In [191]:
import pandas as pd
from dask import dataframe as dd
from dask_ml.linear_model import LinearRegression
from dask_ml.preprocessing import Categorizer, DummyEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from dask_ml.metrics import mean_squared_error
from dask_ml.metrics import r2_score
import dask_ml
from scipy import stats
from distributed import Client, progress

In [159]:
#client = Client()
#client

In [192]:
ddf=dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"])

In [193]:
ddf = ddf.rename(
    columns={
        "weathersit": "weather",
        "mnth": "month",
        "hr": "hour",
        "hum": "humidity",
        "cnt": "count",
        "yr": "year",
    })

In [194]:
ddf = ddf.drop(["dteday"], axis=1)

In [195]:
cats= ["month","hour","holiday","weekday","workingday","weather"]

pipeline = make_pipeline(Categorizer(columns=cats), DummyEncoder(),MinMaxScaler())
ddf_onehot=pipeline.fit_transform(ddf)


Fix  skewness attempt

In [196]:
def correct_skewness(columns=None, max_skewness=0.5):
    if columns is None:
        raise ValueError(
            f"columns argument is None. Please set columns argument to a list of columns"
        )


    for col in columns:
        skewness = stats.skew(ddf_onehot[col])
        max_val = ddf_onehot[col].max().compute()
        min_val = ddf_onehot[col].min().compute()

        if abs(skewness) > max_skewness and (max_val > 1 or min_val < 0):
            delta = 1.0
            if min_val < 0:
                delta = max(1, -min_val + 1)
            ddf_onehot[col] = da.log(delta + ddf_onehot[col])
    return ddf_onehot


In [197]:
correct_skewness(columns=["season",'instant','year','temp','atemp','humidity','windspeed'])

,instant,season,year,temp,atemp,humidity,windspeed,casual,registered,count,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,holiday_0,holiday_1,weekday_6,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [198]:
ddf_onehot.head()

,instant,season,year,temp,atemp,humidity,windspeed,casual,registered,count,...,weekday_2,weekday_3,weekday_4,weekday_5,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4
0,0.000000,0.0,0.0,0.224490,0.2879,0.81,0.0,0.008174,0.014673,0.015369,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.000058,0.0,0.0,0.204082,0.2727,0.80,0.0,0.021798,0.036117,0.039959,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.000115,0.0,0.0,0.204082,0.2727,0.80,0.0,0.013624,0.030474,0.031762,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.000173,0.0,0.0,0.224490,0.2879,0.75,0.0,0.008174,0.011287,0.012295,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.000230,0.0,0.0,0.224490,0.2879,0.75,0.0,0.000000,0.001129,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [185]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X.loc[0:train_size-1], #dask uses loc instead of iloc, and -1 to avoid repeating in both train and test
        X.loc[train_size : len(dataset)],
        y.loc[0:train_size-1], #dask uses loc instead of iloc, and -1 to avoid repeating in both train and test
        y.loc[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [199]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(ddf_onehot, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(ddf_onehot, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(ddf_onehot, "count")

## Converting to arrays


The model does not support dask dataframes, so I will try converting all df to dask arrays by using the .values method.

However, we need to drop 2 columns for each of the count, casual and regular; since dask arrays do not have a .drop method, it will be necessary to do it individually and creating 3 different set of arrays.

### Count

In [200]:
X_train_count = x_train_count.drop(["casual", "registered"], axis = 1)
X_test_count = x_test_count.drop(["casual", "registered"], axis = 1)

In [201]:
X_train_count, X_test_count, \
y_train_count, y_test_count = \
X_train_count.values, X_test_count.values, \
y_train_count.values, y_test_count.values

In [202]:
def score_lin(X_train, X_test, y_train, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)
    print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(y_test, y_pred)))
    print("Variance score (R2): {:.2f}".format(r2_score(y_test, y_pred)))
    return y_pred

In [203]:
baseline_count_pred = score_lin(X_train_count, X_test_count, y_train_count, y_test_count)

/Users/marcosberges/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
/Users/marcosberges/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Intercept: -0.05003191626893421
Coefficients: [ 4.13899824e-02  8.44327568e-02  4.21650787e-02  4.82936528e-02
  1.08148349e-01  9.95946112e-02 -8.35046557e-02 -2.64381016e-02
 -1.23887813e-02 -9.79472559e-03  1.01071293e-02  1.69916761e-02
  3.22617638e-02  1.07871597e-02 -2.38232966e-02 -7.66883926e-03
  1.64430029e-02  1.80081068e-03 -1.67667833e-02 -2.00946366e-02
 -1.22797848e-01 -1.40668284e-01 -1.49043361e-01 -1.60437453e-01
 -1.63823468e-01 -1.47322465e-01 -8.88303953e-02  4.40565695e-02
  1.79806676e-01  3.66812472e-02 -1.52027232e-02  9.69793343e-03
  4.77412154e-02  4.37671141e-02  2.83146046e-02  3.62202763e-02
  9.77277611e-02  2.54623977e-01  2.26484676e-01  1.19848277e-01
  3.88628976e-02 -1.09685151e-02 -4.90277108e-02 -8.97339652e-02
  1.19304912e-02 -7.03535165e-03  7.16586577e-03 -5.18579934e-03
 -2.73709042e-03 -1.43558652e-04  1.69799024e-03  1.68964707e-03
  5.34686274e-03 -4.21648017e-04  2.11840590e-03  1.44737832e-02
  4.87393703e-03 -4.78524655e-02]


/Users/marcosberges/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


Mean squared error (MSE): 0.02


ValueError: cannot convert float NaN to integer

In [15]:
baseline_count_pred.compute()

/Users/marcosberges/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
/Users/marcosberges/anaconda3/lib/python3.6/site-packages/dask/blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


array([338.0738301 , 259.17952237, 278.96130663, ..., 161.79918199,
       130.3656705 ,  83.29175121])